In [3]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.7 MB/s eta 0:00:00:00:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import openpyxl
import re
from tqdm import tqdm
import os

In [5]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import WhitespaceTokenizer
# from nltk.tokenize import WordPunctTokenizer

from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
from nltk.tag import pos_tag

from nltk.stem import WordNetLemmatizer
from nltk import ne_chunk

In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /Users/kimmonica/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/kimmonica/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/kimmonica/nltk_data...
[nltk_data]   Pa

True

### def

In [7]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    try:
        text = text.replace(".", " ").strip() 
        text = text.replace("·", " ").strip() 
        pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
        text = re.sub(pattern=pattern, repl='', string=text)
    except:
        text = '숫자없음' 
    return text 

In [8]:
def clean_words(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    try:
        text = text.replace(".", " ").strip() 
        text = text.replace("·", " ").strip() 
        pattern = '[^0-9]+' 
        text = re.sub(pattern=pattern, repl='', string=text) 
        text = int(text)
    except:
        text = '숫자없음'
    return text 

In [9]:
stop_words = set(stopwords.words('english')) 

#### dataset

In [ ]:
# df_path = '/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie/dataset/'
# file_list = os.listdir(df_path)
# file_list_py = [file for file in file_list if file.endswith('.xlsx')]

# df = pd.DataFrame()
# for i in file_list_py:
#     data = pd.read_excel(df_path + i)
#     df = pd.concat([df,data])
    
# df = df.reset_index(drop = True)

In [19]:
# df = df.drop(columns='Unnamed: 0')
# df.head()

,movie_cnt,movie_title,age,reviews
0,245번,Andi Mack,Child,-Why do most of the kids in my school like thi...
1,245번,Andi Mack,Child,-that show is rated TV-MA
2,245번,Andi Mack,Child,"-I haven't even watched the first episodes, bu..."
3,245번,Andi Mack,Child,"-Brah, this is possibly the worst TV show I ha..."
4,245번,Andi Mack,Child,-So everyone is saying that everything about t...


In [108]:
# df.to_csv('org_reviews.csv')

In [109]:
/pwd

'/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie'

#### data loading

In [18]:
data = pd.read_csv('/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie/org_reviews.csv', index_col=0)

In [19]:
data.head()

,movie_cnt,movie_title,age,reviews
0,245번,Andi Mack,Child,-Why do most of the kids in my school like thi...
1,245번,Andi Mack,Child,-that show is rated TV-MA
2,245번,Andi Mack,Child,"-I haven't even watched the first episodes, bu..."
3,245번,Andi Mack,Child,"-Brah, this is possibly the worst TV show I ha..."
4,245번,Andi Mack,Child,-So everyone is saying that everything about t...


In [20]:
data['age'].value_counts()

Child                      94647
++++Child                  57020
Teen, 13 years old         21511
++++Teen, 13 years old     12002
Teen, 14 years old         11268
Kid, 11 years old           9678
Kid, 12 years old           8483
Kid, 10 years old           7258
++++Teen, 14 years old      6312
Teen, 15 years old          6066
++++Kid, 11 years old       5614
Kid, 9 years old            4928
++++Kid, 12 years old       4527
++++Kid, 10 years old       4280
++++Teen, 15 years old      3091
++++Kid, 9 years old        2811
Kid, 8 years old            2608
Teen, 16 years old          2365
Teen, 17 years old          1605
++++Kid, 8 years old        1565
++++Teen, 16 years old      1229
Kid, 7 years old             995
++++Teen, 17 years old       691
++++Kid, 7 years old         688
Kid, 6 years old             498
++++Kid, 6 years old         377
Adult, 18 years old          185
Kid, 0 years old             170
++++Kid, 0 years old         136
++++Adult, 18 years old      122
Kid, 5 yea

In [21]:
data['age'] = data['age'].apply(clean_text)
data['age'] = data['age'].apply(clean_words )

In [22]:
data= data[data['age']!='숫자없음']
data['age'].value_counts()

13    33513
14    17580
11    15292
12    13010
10    11538
15     9157
9      7739
8      4173
16     3594
17     2296
7      1683
6       875
18      307
0       306
5       130
2       110
3        80
4        74
1         8
Name: age, dtype: int64

In [23]:
data['age'] = data['age'].astype(int)

In [25]:
print(data.shape)
data = data.drop_duplicates()
print(data.shape)

(121465, 4)
(72075, 4)


In [26]:
data['age'].value_counts()

13    19916
14    10403
11     9151
12     7636
10     6975
15     5309
9      4564
8      2553
16     2108
17     1364
7       954
6       504
18      201
0       189
2        74
5        73
3        50
4        47
1         4
Name: age, dtype: int64

In [28]:
data.to_csv('adult_child_org1.csv')

### NEW

In [31]:
data = pd.read_csv('/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie/adult_child_org1.csv', index_col=0)
data.head()

,movie_cnt,movie_title,age,reviews
587,264번,Ant-Man,7,-Ant man is a great marvel movie for younger a...
588,264번,Ant-Man,7,-Not even appropriate for adults. A man brags ...
589,264번,Ant-Man,7,-Ant man is just a great movie and is quite fa...
590,264번,Ant-Man,7,"-Family friendly, unless you consider people g..."
591,264번,Ant-Man,7,-I think it's a really good movie for ages eig...


In [32]:
data.age.value_counts()

13    19916
14    10403
11     9151
12     7636
10     6975
15     5309
9      4564
8      2553
16     2108
17     1364
18     1142
7       954
Name: age, dtype: int64

#### preprocessing

In [33]:
data.head()

,movie_cnt,movie_title,age,reviews
587,264번,Ant-Man,7,-Ant man is a great marvel movie for younger a...
588,264번,Ant-Man,7,-Not even appropriate for adults. A man brags ...
589,264번,Ant-Man,7,-Ant man is just a great movie and is quite fa...
590,264번,Ant-Man,7,"-Family friendly, unless you consider people g..."
591,264번,Ant-Man,7,-I think it's a really good movie for ages eig...


##### sentence dividing

In [34]:
def sent_number(text):
    try:
        num = len(sent_tokenize(text))
    except:
        num = '오류'
    return num

In [35]:
data.shape

(72075, 4)

In [36]:
data['sent_number'] = data['reviews'].apply(sent_number)
data.head()

,movie_cnt,movie_title,age,reviews,sent_number
587,264번,Ant-Man,7,-Ant man is a great marvel movie for younger a...,4
588,264번,Ant-Man,7,-Not even appropriate for adults. A man brags ...,3
589,264번,Ant-Man,7,-Ant man is just a great movie and is quite fa...,1
590,264번,Ant-Man,7,"-Family friendly, unless you consider people g...",5
591,264번,Ant-Man,7,-I think it's a really good movie for ages eig...,1


In [37]:
data['reviews'] = data['reviews'].apply(clean_text)

In [34]:
data.head()

,movie_cnt,movie_title,age,reviews,sent_number
574,264번,Ant-Man,8,I personally enjoyed this movie It was funny ...,3
575,264번,Ant-Man,8,Clever funny cool Marvel movie that has lots o...,1
576,264번,Ant-Man,8,A vary good movie It is a superhero movie so ...,3
577,264번,Ant-Man,8,Antman is funny and a little violent The viol...,3
578,264번,Ant-Man,8,I watched Ant Man in theatres in Switzerland w...,13


In [38]:
data.to_csv('[230625]child_reviews_sentnum.csv')

In [39]:
full_result = []
for i in tqdm(data.reviews.to_list()):
    tokenizer = WhitespaceTokenizer()
    word_tokens = tokenizer.tokenize(i)
    
    result= []
    for word in word_tokens:
        # word = word.lower()
        if word not in stop_words:
            lemm = WordNetLemmatizer()
            word = lemm.lemmatize(word, pos = 'v')
            result.append(word)
    
    full_result.append(result)
    


100%|██████████| 72075/72075 [00:07<00:00, 9817.87it/s] 


In [40]:
full_pos = []
for for_pos in tqdm(full_result):
        k = pos_tag(for_pos)
        ner_sentence = ne_chunk(k) ## NER 개체명인식
        full_pos.append(ner_sentence)

100%|██████████| 72075/72075 [04:44<00:00, 253.60it/s]


In [41]:
print(len(full_pos))

72075


In [42]:
data.shape

(72075, 5)

In [43]:
data['full_pos'] = full_pos
data.head()

,movie_cnt,movie_title,age,reviews,sent_number,full_pos
587,264번,Ant-Man,7,Ant man is a great marvel movie for younger an...,4,"[[(Ant, NNP)], (man, NN), (great, JJ), (marvel..."
588,264번,Ant-Man,7,Not even appropriate for adults A man brags a...,3,"[(Not, RB), (even, RB), (appropriate, JJ), (ad..."
589,264번,Ant-Man,7,Ant man is just a great movie and is quite fam...,1,"[[(Ant, NNP)], (man, NN), (great, JJ), (movie,..."
590,264번,Ant-Man,7,Family friendly unless you consider people get...,5,"[(Family, RB), (friendly, JJ), (unless, IN), (..."
591,264번,Ant-Man,7,I think its a really good movie for ages eight...,1,"[(I, PRP), (think, VBP), (really, RB), (good, ..."


In [44]:
final_pos =[]

for i in tqdm(full_pos):
    try:
        if len(i)>=0:
            test = str(i)
            test = test.replace('\n','')
            # print(test)

            test_list= test.split('  ')
            result= []
            for i in test_list:
                
                if i[1].isupper()==True:
                    pass
                else:
                    result.append(i)
            org_result = []
            for i in result:
                text = str(i)
                text = text.split('/')[0]
                word = text.lower()
                if word not in stop_words:
                    lemm = WordNetLemmatizer()
                    word = lemm.lemmatize(word, pos = 'v')

                    # if len(word)>2:
                    org_result.append(word)
            final_pos.append(org_result)
    except:
        final_pos.append('passed')
        pass

print(len(final_pos))

100%|██████████| 72075/72075 [00:07<00:00, 10046.03it/s]

72075


In [45]:
data['final_pos'] = final_pos

In [46]:
data.to_csv('[230625]fianl_pos.csv')

In [47]:
final_pos_tag = []

for i in tqdm(final_pos):
    try:
        i = pos_tag(i)
        final_pos_tag.append(i)
    except:
        i = '오류'
        final_pos_tag.append(i)
        pass

print(len(final_pos_tag))

100%|██████████| 72075/72075 [00:58<00:00, 1226.61it/s]

72075


In [48]:
data['final_pos_tag'] = final_pos_tag

In [49]:
data.to_csv('[230625]final_pos_tag.csv')

### NEW

In [67]:
import konlpy

In [66]:
from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()

JVMNotFoundException: No JVM shared library file (libjli.dylib) found. Try setting up the JAVA_HOME environment variable properly.

In [59]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 
    
def get_nouns(tokenizer, sentence): 
    """ 단어의 길이가 2이상인 일반명사(NNG),
     고유명사(NNP), 외국어(SL)만을 반환한다. 
     :param tokenizer: 
     :param sentence: :return: """ 
    tagged = tokenizer.nouns(sentence)
    nouns = [s for s in tagged if len(s)>1] 
     
    return nouns 
     
     
def tokenize(df): 
    tokenizer = okt
    processed_data = [] 
    for sent in tqdm(df['reviews']):
        sentence = clean_text(sent.replace('\n', '').strip()) 
        processed_data.append(get_nouns(tokenizer, sentence)) 
        
    return processed_data
    
    
def save_processed_data(processed_data): 
    """ 토큰 분리한 데이터를 csv로 저장 :param processed_data: :return: """ 
    
    with open('lsa_token_pos.csv', 'w', newline='', encoding='utf-8') as f: 
        writer = csv.writer(f) 
        for data in processed_data:
             writer.writerow(data)

In [54]:
data.head()

,movie_cnt,movie_title,age,reviews,sent_number,full_pos,final_pos,final_pos_tag
587,264번,Ant-Man,7,Ant man is a great marvel movie for younger an...,4,"[[(Ant, NNP)], (man, NN), (great, JJ), (marvel...","[man, great, marvel, movie, younger, older, ki...","[(man, NN), (great, JJ), (marvel, JJ), (movie,..."
588,264번,Ant-Man,7,Not even appropriate for adults A man brags a...,3,"[(Not, RB), (even, RB), (appropriate, JJ), (ad...","[even, appropriate, adults, man, brag, touch, ...","[(even, RB), (appropriate, JJ), (adults, NNS),..."
589,264번,Ant-Man,7,Ant man is just a great movie and is quite fam...,1,"[[(Ant, NNP)], (man, NN), (great, JJ), (movie,...","[man, great, movie, quite, family, friendly]","[(man, NN), (great, JJ), (movie, NN), (quite, ..."
590,264번,Ant-Man,7,Family friendly unless you consider people get...,5,"[(Family, RB), (friendly, JJ), (unless, IN), (...","[family, friendly, unless, consider, people, g...","[(family, NN), (friendly, RB), (unless, IN), (..."
591,264번,Ant-Man,7,I think its a really good movie for ages eight...,1,"[(I, PRP), (think, VBP), (really, RB), (good, ...","[think, really, good, movie, age, eight, reall...","[(think, VB), (really, RB), (good, JJ), (movie..."


In [56]:
for sent in tqdm(data['reviews']):
    try:
        sentence = clean_text(sent.replace('\n', '').strip())
        # print(sentence)
    except:
        print(sent)
        pass

100%|██████████| 72075/72075 [00:00<00:00, 206022.24it/s]


In [62]:
from konlpy.tag import Okt

In [64]:
## 오래걸리니까 조심

if __name__ == '__main__':
     # df = pd.read_csv('negative.csv', index_col = 0)
     processed_data = tokenize(data) # description 부분을 토크나이징 한다. 
     
     # 토큰 분리한 데이터를 저장 
     save_processed_data(processed_data)


NameError: name 'okt' is not defined

##### ne_chunk로 NER한 후, 재tagging

In [48]:
data.head()

,movie_cnt,movie_title,age,reviews,sent_number,full_pos,final_pos,final_pos_tag
574,264번,Ant-Man,8,I personally enjoyed this movie It was funny ...,3,"[(I, PRP), (personally, RB), (enjoy, VBP), (mo...","[personally, enjoy, movie, funny, plot, decent...","[(personally, RB), (enjoy, JJ), (movie, NN), (..."
575,264번,Ant-Man,8,Clever funny cool Marvel movie that has lots o...,1,"[[(Clever, NNP)], (funny, JJ), (cool, NN), [(M...","[funny, cool, movie, lot, scifi, action]","[(funny, JJ), (cool, JJ), (movie, NN), (lot, N..."
576,264번,Ant-Man,8,A vary good movie It is a superhero movie so ...,3,"[(A, DT), (vary, JJ), (good, JJ), (movie, NN),...","[vary, good, movie, superhero, movie, theres, ...","[(vary, JJ), (good, JJ), (movie, NN), (superhe..."
577,264번,Ant-Man,8,Antman is funny and a little violent The viol...,3,"[[(Antman, NNP)], (funny, VBD), (little, JJ), ...","[funny, little, violent, violence, pretty, sta...","[(funny, JJ), (little, JJ), (violent, JJ), (vi..."
578,264번,Ant-Man,8,I watched Ant Man in theatres in Switzerland w...,13,"[(I, PRP), (watch, VBP), [(Ant, NNP), (Man, NN...","[watch, theatres, brother, mom, review, mine, ...","[(watch, NN), (theatres, VBZ), (brother, JJR),..."


In [49]:
data.final_pos_tag.tolist()[0]

[('personally', 'RB'),
 ('enjoy', 'JJ'),
 ('movie', 'NN'),
 ('funny', 'JJ'),
 ('plot', 'NN'),
 ('decent', 'NN'),
 ('thankfully', 'RB'),
 ('probably', 'RB'),
 ('others', 'NNS'),
 ('wasnt', 'RB'),
 ('much', 'JJ'),
 ('term', 'NN'),
 ('sexual', 'JJ'),
 ('innuendos', 'NN'),
 ('kiss', 'JJ'),
 ('comment', 'NN'),
 ('relate', 'NN'),
 ('topic', 'NN'),
 ('overall', 'JJ'),
 ('wasnt', 'NN'),
 ('bad', 'JJ'),
 ('iron', 'NN'),
 ('man', 'NN')]

In [241]:
import nltk

# Example sentence
sentence = "Apple is looking at buying U.K. startup for $1 billion"

# Tokenize the sentence
tokens = nltk.word_tokenize(sentence)

# Perform part-of-speech tagging
pos_tags = nltk.pos_tag(tokens)

# Perform named entity recognition
chunked = nltk.ne_chunk(pos_tags)

# Remove named entities from the original sentence
filtered_sentence = ' '.join([w[0] for w in pos_tags if not hasattr(w, 'label')])

print(filtered_sentence)

Apple is looking at buying U.K. startup for $ 1 billion


### RRE

In [51]:
df = pd.read_csv('/Users/kimmonica/Documents/GitHub/Text_Mining/common_sense_movie/[230617]child_data.csv')
df.head()

,Unnamed: 0,movie_cnt,movie_title,age,reviews
0,574,264번,Ant-Man,8,-I personally enjoyed this movie. It was funny...
1,575,264번,Ant-Man,8,"-Clever, funny, cool Marvel movie that has lot..."
2,576,264번,Ant-Man,8,-A vary good movie. It is a superhero movie so...
3,577,264번,Ant-Man,8,-Ant-man is funny and a little violent. The vi...
4,578,264번,Ant-Man,8,-I watched Ant Man in theatres in Switzerland ...


In [52]:
data = df[['age', 'reviews']]
print(data.shape)
data = data.drop_duplicates()
print(data.shape)
data.to_csv('final_use.csv')

(7015, 2)
(2231, 2)
